In [1]:
!pip install surprise

     |████████████████████████████████| 11.8MB 255kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670949 sha256=78c8cd0ac08719bae2edeee86d9818da5a3b969cc3b2a8dadcedf25829a8ef24
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
import io, os, json
from surprise import Dataset ,Reader, accuracy, SVD, dump
import pandas as pd
import numpy as np
from itertools import chain

In [3]:
# 노래 추천 결과에 이용할 data

song_meta = pd.read_pickle("/content/drive/My Drive/recommendation sys/song_meta_0802.pkl") # new_tags와 cluster_id가 들어있는 song_meta
song_frequency_cluster = pd.read_pickle("/content/drive/My Drive/recommendation sys/추천프로젝트/song_frequency_cluster.pkl") # cluster별로 속한 노래들을 빈도 내림차순으로 정리한 df

def func(x):
  songs = x.id.to_numpy()
  arg_mask = np.argsort(x.frequency.to_numpy())[::-1]
  return songs[arg_mask]

cluster = song_frequency_cluster.groupby('cluster').apply(func).to_dict() # cluster별로 song_id가 빈도 내림차순으로 정리된 dict

In [4]:
# id_to_item dictionary

tags = pd.unique(list(chain.from_iterable(song_meta.new_tags)))
sid_to_cid = dict(zip(song_meta.id.map(str), song_meta.new_song_id.map(str)))

n_clusters = 1000
n_tags = 110

tag_to_tid = {tags[i]:str(n_clusters + i) for i in range(len(tags))}
tid_to_tag = {tid:tag for tag,tid in tag_to_tid.items()}
sid_to_song = song_meta.song_name.to_dict()

In [122]:
# validation set
# 속한 노래와 태그를 id로 mapping한 후 합쳐서 cluster_tags에 list형태로 저장

val = pd.read_json("/content/drive/My Drive/recommendation sys/0802/val.json")
val['cid'] = val.songs.map(lambda x:[sid_to_cid[str(i)] for i in x])
val['tid'] = val.tags.map(lambda x:[tag_to_tid[i] for i in x if i in tags])
val['clusters_tags'] = val.cid + val.tid
val

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000,"[673, 673, 55, 833, 776, 424, 472, 666, 234, 7...",[],"[673, 673, 55, 833, 776, 424, 472, 666, 234, 7..."
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,[],[],[]
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000,"[166, 722, 285, 285, 290, 937, 910, 220, 726, ...",[],"[166, 722, 285, 285, 290, 937, 910, 220, 726, ..."
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000,"[655, 493, 213, 956, 145, 510, 145, 485, 857, ...",[],"[655, 493, 213, 956, 145, 510, 145, 485, 857, ..."
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000,"[605, 940, 764, 764, 751, 584, 940, 584]",[],"[605, 940, 764, 764, 751, 584, 940, 584]"
...,...,...,...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000,"[804, 194, 935, 87, 655, 133, 857, 803, 523, 9...",[1036],"[804, 194, 935, 87, 655, 133, 857, 803, 523, 9..."
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000,"[377, 834, 418, 613, 295, 152, 787, 489, 286, ...",[],"[377, 834, 418, 613, 295, 152, 787, 489, 286, ..."
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000,"[857, 857, 857, 857, 857, 857, 857, 857, 857, ...",[],"[857, 857, 857, 857, 857, 857, 857, 857, 857, ..."
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000,"[531, 615, 155, 347, 70, 347, 347, 615, 618]",[],"[531, 615, 155, 347, 70, 347, 347, 615, 618]"


In [8]:
# prediction에 이용할 모델

algo = pd.read_pickle("/content/drive/My Drive/recommendation sys/0802/algo_0802.pkl")
train_obj = pd.read_pickle("/content/drive/My Drive/recommendation sys/0802/train_obj_0802.pkl")
user_matrix = algo.pu
item_matrix = algo.qi
raw_to_inner = {int(train_obj.to_raw_iid(i)):i for i in range(n_clusters+n_tags)}

# Prediction (detail)
> Prediction 과정을 자세히 설명합니다.

In [223]:
val.sort_values('tid').iloc[-4000:-3950]

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
3601,"[카페, 잔잔한, 힐링, 알엔비]",86486,,"[344931, 327474, 649662, 236514, 196984, 49725...",9,2019-12-11 21:56:47.000,"[750, 189, 512, 99, 679, 359, 237, 358, 359, 189]","[1033, 1036, 1022]","[750, 189, 512, 99, 679, 359, 237, 358, 359, 1..."
18577,"[따듯한, 카페, 잔잔한, 사랑, 퇴근길]",134469,커피와 함께 하는 감성가득한 노래,[],0,2019-12-18 07:57:39.000,[],"[1033, 1036, 1046, 1101]","[1033, 1036, 1046, 1101]"
211,"[떠나버려, 카페, 잔잔한, 이별, 왜그랬을까]",90469,,"[15658, 231659, 417870, 508608, 567078, 663905...",6,2019-04-24 09:20:19.000,"[655, 133, 890, 133, 451, 804, 680, 798, 377, ...","[1033, 1036, 1047]","[655, 133, 890, 133, 451, 804, 680, 798, 377, ..."
19303,"[카페, 잔잔한, 일상]",94351,,"[416678, 235522, 146922, 234999, 399907, 16116...",4,2017-02-07 11:40:25.000,"[97, 273, 968, 804, 273, 47, 873, 99, 661, 212...","[1033, 1036, 1055]","[97, 273, 968, 804, 273, 47, 873, 99, 661, 212..."
22113,"[카페, 잔잔한, 매장, 사랑, 감사]",56469,"5월, 사랑과 감사의 마음을 담아 선곡한 연주곡",[],1,2017-05-04 09:47:45.000,[],"[1033, 1036, 1103, 1046]","[1033, 1036, 1103, 1046]"
8779,"[카페, 조용한노래, 분위기, 드라이브, 여행]",14294,,"[289019, 705815, 78180, 368429, 168607, 646927...",6,2017-08-14 11:22:18.000,"[804, 266, 194, 194, 846, 890, 798, 954, 804, ...","[1033, 1037, 1001, 1006]","[804, 266, 194, 194, 846, 890, 798, 954, 804, ..."
13714,"[카페, 분위기, 드라이브, 사랑]",57140,,"[384299, 341513, 402463, 397334, 698536, 30009...",2,2018-02-28 08:44:53.000,"[804, 485, 928, 680, 804, 145, 857, 857, 890, ...","[1033, 1037, 1001, 1046]","[804, 485, 928, 680, 804, 145, 857, 857, 890, ..."
2402,"[카페, 분위기, 커피]",125642,,"[364111, 124836, 208962, 513419, 418935, 55211...",6,2017-10-31 12:20:22.000,"[626, 125, 935, 79, 145, 882, 857, 523, 578, 3...","[1033, 1037, 1096]","[626, 125, 935, 79, 145, 882, 857, 523, 578, 3..."
11684,"[로맨틱, 카페, 분위기, 피아노]",15174,,"[634348, 51035, 303716, 342786, 150323, 528279...",11,2017-09-19 01:04:47.000,"[842, 557, 967, 842, 595, 76, 557, 53, 53, 744...","[1033, 1037, 1108]","[842, 557, 967, 842, 595, 76, 557, 53, 53, 744..."
9275,"[겨울감성, 카페, 인디음악, 쓸쓸, 눈]",147252,분위기 좋은 인디가수모음집,[],18,2017-03-16 12:10:48.000,[],"[1033, 1039]","[1033, 1039]"


In [236]:
############# 1. SVD 행렬을 행 별로 normalize 시킨다.

n_values = n_clusters + n_tags
org = user_matrix.dot(item_matrix.T) # org_matrix
norm = np.linalg.norm(org,axis=1).reshape(-1,1)
SVD_mat_scaled = org/norm


# 결과를 확인하고 싶은 index 지정
# idx = 8086
idx = 1046
target = val.iloc[idx]

# extract clusters/tags 
values = [train_obj.to_inner_iid(str(i)) for i in val.iloc[idx].clusters_tags]

# convert to one-hot vector
s = np.eye(n_values)[values].sum(axis=0)

# calcuate similarity
sim = SVD_mat_scaled.dot(s)
most_similar_ply = sim.argmax() # the most similar plylst

display(val.iloc[[idx]])

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
1046,"[배경음악, 카페, 독서, 뉴에이지, 아침]",102119,,"[439336, 335116, 552555, 17328, 29793, 113724,...",2,2020-01-13 14:24:31.000,"[70, 133, 302, 113, 133, 52, 302, 855, 302, 76...","[1033, 1042, 1017, 1048]","[70, 133, 302, 113, 133, 52, 302, 855, 302, 76..."


In [237]:
# 위에서 추출한 가장 유사한 plylst 의 실제 값 확인해보기
tr.query(f'id=={train_obj.to_raw_uid(most_similar_ply)}')

,tags,id,plylst_title,songs,like_cnt,updt_date
60166,"[카페, 연주곡, 매장, bgm, 독서, 뉴에이지, 가을, 공부, 밤]",137903,가을날 카페에서 틀어주는 피아노 BGM,"[304995, 131690, 645171, 622433, 251746, 64691...",12,2019-01-29 19:13:13.000


In [238]:
#########
# songs #
#########

song_inner_id = [raw_to_inner[i] for i in range(0,1000)]
sim_clusters = SVD_mat_scaled[most_similar_ply,song_inner_id] # 가장 유사한 유저의 노래 점수를 추출

songs = set(target.songs)
len_ = len(songs)

n = 100
end_num = len_ + n

# 점수를 고려하여 cluster별로 노래를 선택합니다.
# ex : cluster 1에 대한 점수가 0.3인 경우 : cluster 1에서 30개의 노래를 빈도 순으로 추출한다.

for ratio, cluster_inner_id in sorted(zip(sim_clusters,song_inner_id), reverse=True):
    cluster_id = int(train_obj.to_raw_iid(cluster_inner_id))
    num = int(n*ratio)*1.3 + 1
    limit =len_ + num

    for song in cluster[cluster_id]:
        end_state = (len(songs) >= end_num)
        if (len(songs) >= limit) or end_state:
            break
        songs.add(str(song))
    
    if end_state:
        break
    len_ = len(songs)

recommended_songs = songs - set(target.songs)

print("Ground truth :", [sid_to_song[i] for i in target.songs])
print("Recommended songs :")
for i in recommended_songs:
    print(sid_to_song[int(i)])

Ground truth : ['Danny Boy', 'Merry Christmas Mr. Lawrence', 'Reminiscent (회상)', '그럴 수 있다면', 'River Flows In You', '다시 돌아온 계절', 'Chaconne', '함께 걷던 길', 'Suddenly', '낯선 하루 (A Strange Day)', '오늘은 왠지...', '이제는 쉬어도 돼', '아픔을 잊는 법', '그대를 생각했던 순간들', '내리던 가을잎은', '그 집', '환상(illusion fantasy)', '위로가 필요한 밤', '놀이터', '기도', '마음정원']
Recommended songs :
Sight Of You
Copenhagen (Feat. Sonyae)
Faure : Dolly Suite Op.56 - I. Berceuse (포레 : 돌리 모음곡 작품번호 56 - 1번. 자장가)
Pains
Lifted (Feat. Cassow)
Barbie Girl
Bad To You
Chopin: Nocturne No. 1 in B-Flat Minor, Op. 9 No. 1
Soon You’ll Get Better (Feat. Dixie Chicks)
Hurt Me (From Songland)
Weak
Do Your Time
Memories (Dillon Francis Remix)
Freak Me 
Fool Of Me
Chopin : Nocturne No.20 In C Sharp Minor Op.Posth (쇼팽 : 야상곡 20번 올림 다단조)
Demon
Fruhlingstraum (봄꿈)
One In A Million
Better Now
Red Hearse
Uptown Girl
Rollercoaster
Hit the Back
Don’t Call Me Angel (Charlie’s Angels)
You Do It
림스키 코르사코프 : 왕벌의 비행
Ain`t the Same
Keep On
Brahms : Waltz In A Flat Major Op.39-15 (

In [239]:
########
# tags #
########

tag_inner_id = np.array([raw_to_inner[i] for i in range(1000,1110)])
sim_tags = SVD_mat_scaled[most_similar_ply,tag_inner_id]

recommended_tags = set()

for _, tag_id in sorted(zip(sim_tags, tag_inner_id), reverse=True):
    recommended_tags.add(train_obj.to_raw_iid(tag_id))
    if len(recommended_tags) == 20:
        break

recommended_tags = [tid_to_tag[i] for i in recommended_tags - set(target.tags)]

print("Ground truth :", target.tags)
print("Recommended tags :",recommended_tags)

Ground truth : ['배경음악', '카페', '독서', '뉴에이지', '아침']
Recommended tags : ['편안한', 'EDM', '록', '독서', '설렘', '연인', '따뜻한', '매장', '연애', '여행', '음색', '저녁', '집중', '스트레스', '여름밤', '산책', '커피', '연주곡', '혼자', '감성']


# Prediction



In [234]:
from tqdm import tqdm

In [167]:
answers = []

In [ ]:
n_values = n_clusters + n_tags
org = user_matrix.dot(item_matrix.T) # org_matrix
norm = np.linalg.norm(org,axis=1).reshape(-1,1)
SVD_mat_scaled = org/norm

In [ ]:
for idx in range(len(val)):

    target = val.iloc[idx]

    # extract clusters/tags
    values = [train_obj.to_inner_iid(str(i)) for i in target.clusters_tags]

    # convert to one-hot vector
    s = np.eye(n_values)[values].sum(axis=0)
    sim = org.dot(s) # sim.argmax() ; 가장 유사한 cluster"

    song_inner_id = [raw_to_inner[i] for i in range(0,1000)]
    sim_cluters = SVD_mat_scaled[sim.argmax(),song_inner_id]

    songs = set(target.songs)
    len_ = len(songs)

    n = 100
    end_num = len_ + n

    for ratio, cluster_id in zip(np.sort(sim_cluters)[::-1], np.argsort(sim_cluters)[::-1]):
        num = int(n*ratio) + 1
        limit =len_ + num

        for song in cluster[cluster_id]:
            end_state = (len(songs) >= end_num)
            if (len(songs) >= limit) or end_state:
                break
            songs.add(str(song))
        
        if end_state:
            break
        len_ = len(songs)

    recommended_songs = songs - set(target.songs)

    tag_inner_id = np.array([raw_to_inner[i] for i in range(1000,1110)]) # 1032,1068,1101
    sim_tags = np.argsort(SVD_mat_scaled[sim.argmax(),tag_inner_id])[::-1]

    recommended_tags = set(target.tags)
    len_ = len(recommended_tags)

    n = 10
    end_num = len_ + n

    for i in tag_inner_id[sim_tags]: # 유사도별 내림차순
        recommended_tags.add(train_obj.to_raw_iid(i))
        if len(recommended_tags) == end_num:
            break

    recommended_tags = [tid_to_tag[i] for i in recommended_tags - set(target.tags)]

    answers.append({
    "id": target.id,
    "songs": list(recommended_songs),
    "tags": list(recommended_tags)
    })

In [ ]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [ ]:
# tag mapping 안됐을 때

a = pd.DataFrame(answers)
a['tags'] = a.tags.map(lambda x: [tid_to_tag[t] for t in x])
a['tags'] = a.tags.map(lambda x: [int(t) for t in x])
a['songs'] = a['songs'].map(lambda x: [int(s) for s in x])
res = list(a.transpose().to_dict().values())

In [ ]:
write_json(answers,"/content/drive/My Drive/recommendation sys/results_0802_split.json")